In [1]:
# Dependencies
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Google API Key
from config import gkey

In [2]:
#target city
target_city = "San Diego, CA"

In [3]:
#endpoint url

target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_city, gkey)

In [4]:
# Run a request to endpoint and convert result to json
sd_data = requests.get(target_url).json()

# Print the json
print(sd_data)

{'results': [{'address_components': [{'long_name': 'San Diego', 'short_name': 'San Diego', 'types': ['locality', 'political']}, {'long_name': 'San Diego County', 'short_name': 'San Diego County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'California', 'short_name': 'CA', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'San Diego, CA, USA', 'geometry': {'bounds': {'northeast': {'lat': 33.114249, 'lng': -116.90816}, 'southwest': {'lat': 32.534856, 'lng': -117.3097969}}, 'location': {'lat': 32.715738, 'lng': -117.1610838}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 33.114249, 'lng': -116.90816}, 'southwest': {'lat': 32.534856, 'lng': -117.3097969}}}, 'place_id': 'ChIJSx6SrQ9T2YARed8V_f0hOg0', 'types': ['locality', 'political']}], 'status': 'OK'}


In [5]:
# Print the json (pretty printed)
print(json.dumps(sd_data, indent=2, sort_keys=True))

{
  "results": [
    {
      "address_components": [
        {
          "long_name": "San Diego",
          "short_name": "San Diego",
          "types": [
            "locality",
            "political"
          ]
        },
        {
          "long_name": "San Diego County",
          "short_name": "San Diego County",
          "types": [
            "administrative_area_level_2",
            "political"
          ]
        },
        {
          "long_name": "California",
          "short_name": "CA",
          "types": [
            "administrative_area_level_1",
            "political"
          ]
        },
        {
          "long_name": "United States",
          "short_name": "US",
          "types": [
            "country",
            "political"
          ]
        }
      ],
      "formatted_address": "San Diego, CA, USA",
      "geometry": {
        "bounds": {
          "northeast": {
            "lat": 33.114249,
            "lng": -116.90816
          },
          

In [6]:
#extract lat/long
# Extract latitude and longitude
lat = sd_data["results"][0]["geometry"]["location"]["lat"]
lng = sd_data["results"][0]["geometry"]["location"]["lng"]

# Print the latitude and longitude
print('''
    City: {0}
    Latitude: {1}
    Longitude: {2}
    '''.format(target_city, lat, lng))


    City: San Diego, CA
    Latitude: 32.715738
    Longitude: -117.1610838
    


In [7]:
#bring in a csv that includes zips for city
unsorted_df = pd.read_csv("age_estimate_by_zip.csv")
unsorted_df.head()

,Unnamed: 0,group_10yr,population,sex,year,zip
0,0,Under 10,1031,Male,2030,91901
1,1,10 to 19,1454,Male,2030,91901
2,2,20 to 29,929,Male,2030,91901
3,3,30 to 39,926,Male,2030,91901
4,4,40 to 49,1214,Male,2030,91901


In [8]:
#extract zips for city
cities_pd = pd.DataFrame(unsorted_df["zip"].unique())
cities_pd



,0
0,91901
1,91902
2,91905
3,91906
4,91910
5,91911
6,91913
7,91914
8,91915
9,91916


In [9]:
cities_pd.to_csv('san_diego_zips.csv', sep=',')


In [10]:
cities_pd = cities_pd.rename(columns = {0: "Zip Code"}) 
cities_pd
len(cities_pd)

113

In [11]:
# Add columns for lat, lng, business name, business rating
# Note that we used "" to specify initial entry.
cities_pd["Lat"] = ""
cities_pd["Lng"] = ""
cities_pd["Florists"] = ""
cities_pd["Spa"] = ""
cities_pd["Spa Avg Rating"] = ""
cities_pd["Resturaunts Avg Rating"] = ""
cities_pd["Synagogue"] = ""
cities_pd["Veterinary_Care"] = ""
cities_pd["Church"] = ""
cities_pd["Mosque"] = ""
cities_pd["Supermarket"] = ""
cities_pd["Synagogue"] = ""
cities_pd["Bicycle Store"] = ""
cities_pd.head()

,Zip Code,Lat,Lng,Florists,Spa,Spa Avg Rating,Resturaunts Avg Rating,Synagogue,Veterinary_Care,Church,Mosque,Supermarket,Bicycle Store
0,91901,,,,,,,,,,,,
1,91902,,,,,,,,,,,,
2,91905,,,,,,,,,,,,
3,91906,,,,,,,,,,,,
4,91910,,,,,,,,,,,,


In [12]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the cities_pd and run a lat/long search for each zip code
for index, row in cities_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    zip_code = row['Zip Code']
   

    # update address key value
    params['address'] = f"San Diego, CA {zip_code}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()

    cities_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    cities_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
cities_pd.head()

,Zip Code,Lat,Lng,Florists,Spa,Spa Avg Rating,Resturaunts Avg Rating,Synagogue,Veterinary_Care,Church,Mosque,Supermarket,Bicycle Store
0,91901,32.8363,-116.726,,,,,,,,,,
1,91902,32.6704,-117.015,,,,,,,,,,
2,91905,32.6811,-116.301,,,,,,,,,,
3,91906,32.6805,-116.49,,,,,,,,,,
4,91910,32.6385,-117.062,,,,,,,,,,


In [19]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "florist", 
    #"keyword": "",
    "key": gkey
}

record = 1

# Use the lat/lng we recovered to identify establishments
for index, row in cities_pd.iterrows():
   
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    cities_response = requests.get(base_url, params=params).json()
    
    florists = []
    for coor in cities_response["results"]:
        store_type = coor["types"]
        if len(store_type) == 1:
            florists.append(coor)
    print(florists)
            
 
   
    '''try:
        row["Florist"] = cities_response[index,"sys"]["country"]
        '''

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


KeyboardInterrupt: 

In [18]:
florists

[]

In [15]:
print(json.dumps(cities_response, indent=2, sort_keys=True))

{
  "html_attributions": [],
  "next_page_token": "CqQCHwEAALgBM4KasQ6KaBM6l-SrD3IQBcBPkPg91RFNzARMKkAkWB5jU58X0WKQlxVWwrhjXDiCJ7pMRX3ccBOHKef2XidNyDthxo28qkVn3m4yA6zro7ufzPhyKilw0EX6_caTkyUjZeSVGit7lSQW0_PDIL3ojViGkvmXfUg3CJpkDEdxig8eQ333xMmNG2j_Ce287FMN6Xt8PNuqo4XwHtOvd9qlroSvZKlknm-3BWpicrgalkURT6wK4eBx9b5yBTwiFPoU64hw7wAxr0Ab5vnvra1SaM3cZzEZRAB0bKydqpl9P4IeRyUxLSthke8MjelC7hcWqKMKacwQEvFUu7kCtWM_KlzQPe1c7xmhLS2MfV2avF-gWoAmYZpnVIlufYzmiRIQPRaByKjz_JTdLTarpo3EKRoU4rgdx9AexY2xijUDbnG6JPzRSZM",
  "results": [
    {
      "geometry": {
        "location": {
          "lat": 33.6504266,
          "lng": -117.8375632
        },
        "viewport": {
          "northeast": {
            "lat": 33.65168843029149,
            "lng": -117.8361980697085
          },
          "southwest": {
            "lat": 33.64899046970849,
            "lng": -117.8388960302915
          }
        }
      },
      "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png",
      "id": "6c

In [14]:
 try:


        #city.append(response["name"])
        row["Florist"] = cities_response[index,"sys"]["country"]
        '''#row["City ID"] = cities_response["sys"]["id"]
        row["Latitude"] = cities_response["coord"]["lat"]
        row["Longitude"] = cities_response["coord"]["lon"]
        row['Temperature (F)'] = cities_response["main"]["temp_max"]
        row['Humidity (%)'] = cities_response["main"]["humidity"]
        row['Cloudiness (%)'] = cities_response["clouds"]["all"]
        row['Wind Speed (mph)'] = cities_response["wind"]["speed"]
'''



        record += 1 



        #print(city+"&id="+str(cities_url))
        print(cities_url)


    except:
        print(f"Data is missing, skipping")
        continue
    
    cities_pd.head()
   

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 25)

In [ ]:
'''cities_lat_lng = cities_lat_lng.json()

    cities_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    cities_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

'''

In [ ]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": len("florist"),
    #"keyword": "international airport",
    "key": gkey
}

record = 1

# Use the lat/lng we recovered to identify establishments
for index, row in cities_pd.iterrows():
   
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    try:

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    cities_response = requests.get(base_url, params=params)
    
    


        #city.append(response["name"])
        row["Florist"] = cities_response["sys"]["country"]
        '''#row["City ID"] = cities_response["sys"]["id"]
        row["Latitude"] = cities_response["coord"]["lat"]
        row["Longitude"] = cities_response["coord"]["lon"]
        row['Temperature (F)'] = cities_response["main"]["temp_max"]
        row['Humidity (%)'] = cities_response["main"]["humidity"]
        row['Cloudiness (%)'] = cities_response["clouds"]["all"]
        row['Wind Speed (mph)'] = cities_response["wind"]["speed"]
'''



        record += 1 



        #print(city+"&id="+str(cities_url))
        print(cities_url)


    except:
        print(f"Data is missing, skipping")
        continue
    
    cities_pd.head()
   

In [ ]:
row["Florists"] = cities_response["sys"]["country"]

In [ ]:
print(f"Beginning Data Retrieval")
print(f"-------------------------------")



record = 1

for index, row in cities_pd.iterrows():
    #print(row['Florists'])
    try:


        cities_url = query_url + row["Zip Code"]
        cities_response = requests.get(cities_url).json()



        #city.append(response["name"])
        row["Florists"] = cities_response["sys"]["country"]
        '''row["City ID"] = cities_response["sys"]["id"]
        row["Latitude"] = cities_response["coord"]["lat"]
        row["Longitude"] = cities_response["coord"]["lon"]
        row['Temperature (F)'] = cities_response["main"]["temp_max"]
        row['Humidity (%)'] = cities_response["main"]["humidity"]
        row['Cloudiness (%)'] = cities_response["clouds"]["all"]
        row['Wind Speed (mph)'] = cities_response["wind"]["speed"]
'''



        record += 1 



        #print(city+"&id="+str(cities_url))
        print(cities_url)


    except:
        print(f"Data is missing, skipping")
        continue

In [ ]:
''' # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        cities_pd.loc[index, "Airport Name"] = name_address["results"][0]["name"]
        cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")'''